In [2]:
import yfinance as yf
import pandas as pd
import psycopg2
from psycopg2 import sql
from psycopg2.extras import execute_batch
from datetime import datetime
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def download_stock_data(tickers, start_date, end_date):
    data = {}
    for ticker in tickers:
        try:
            df = yf.download(ticker, start=start_date, end=end_date, auto_adjust=False)
            if not df.empty: #if can dl data then add
                data[ticker] = df
            else:
                print(f"No data found for {ticker}")
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
    return data

start_date = '2022-01-01'
end_date = '2022-03-01'

download_stock_data(['PYPL'], start_date, end_date)

[*********************100%***********************]  1 of 1 completed


{'PYPL': Price        Adj Close       Close        High         Low        Open  \
 Ticker            PYPL        PYPL        PYPL        PYPL        PYPL   
 Date                                                                     
 2022-01-03  194.940002  194.940002  196.000000  189.949997  191.410004   
 2022-01-04  191.139999  191.139999  196.100006  188.190002  196.000000   
 2022-01-05  187.160004  187.160004  194.250000  187.149994  189.865005   
 2022-01-06  192.270004  192.270004  195.000000  185.070007  185.520004   
 2022-01-07  187.600006  187.600006  193.125000  186.300003  190.020004   
 2022-01-10  182.949997  182.949997  185.750000  177.399994  185.750000   
 2022-01-11  191.520004  191.520004  192.850006  182.619995  182.990005   
 2022-01-12  187.199997  187.199997  191.449997  185.279999  189.699997   
 2022-01-13  181.009995  181.009995  190.839996  180.350006  187.779999   
 2022-01-14  178.419998  178.419998  182.000000  175.399994  179.149994   
 2022-01-18  174.

In [7]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

def extract_stock_data(tickers, start_date='2022-01-01', end_date='2022-03-01'):
    """Extract stock price data from Alpha Vantage API"""
    # Get API key from environment variable
    api_key = os.getenv("ALPHA_VANTAGE_API_KEY")
    if not api_key:
        raise ValueError("Alpha Vantage API key not found in environment variables")
    
    # Initialize Alpha Vantage client
    ts = TimeSeries(key=api_key, output_format='pandas')
    
    data = {}
    for ticker in tickers:
        try:
            print(f"Downloading {ticker} data from Alpha Vantage...")
            # Get daily adjusted data
            df, meta_data = ts.get_daily_adjusted(symbol=ticker, outputsize='full')
            
            # Alpha Vantage returns data with most recent dates first, so sort by date
            df = df.sort_index()
            
            # Filter the date range
            df = df[(df.index >= start_date) & (df.index <= end_date)]
            
            if not df.empty:
                # Add ticker column
                df['ticker'] = ticker
                data[ticker] = df
                print(f"Successfully downloaded {ticker} data, shape: {df.shape}")
            else:
                print(f"No data found for {ticker} in specified date range")
                
        except Exception as e:
            print(f"Error downloading {ticker}: {e}")
    
    return data

extract_stock_data(["PYPL"])

Error downloading PYPL: Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints


{}